In [1]:
import ipyfan
from ipyfan.utils import norm_fn, rgba2rgb
import numpy as np
import ipywidgets as widgets
import glob
from pathlib import Path
import os
import shutil
import matplotlib.pyplot as plt
from ipyfan.iis_model import ritm_model as iis_model  # AI
from solarium import read_panel_image, load_series

ipyfan.ritm.isegm.model.is_hrnet_model
torch.nn.modules.batchnorm


In [2]:
loc_ind = 3
# rm ann dir
shutil.rmtree("figs/annotations", ignore_errors=True)
data_dir = "/home/franchesoni/data/kayrros/solar/"
locations = os.listdir(data_dir)
print(locations)
print("selected location", locations[loc_ind])
site_zip_Path = Path(data_dir) / locations[loc_ind]
series = load_series(site_zip_Path, outdir='figs')[0]
num_dates = len(series['B02'])

['roseland_i_and_ii.zip', 'bluebell_ii.zip', 'bluebell_ii', 'holstein_1_solar_farm.zip', 'phoebe.zip', 'grizzly_ridge.zip']
selected location holstein_1_solar_farm.zip


In [3]:
validate_button = widgets.Button(
    description='Validate',
    disabled=False)
previous_button = widgets.Button(
    description='Previous',
    disabled=False)
progress = widgets.IntProgress(
    value=0,
    min=0,
    max=num_dates-1,
    description='Progress:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'black'},
    orientation='horizontal'
)

def validate(b):
    global w
    out_path = Path('figs') / 'annotations' / ('date_' + str(progress.value) + '_annotation.png')
    out_path.parent.mkdir(exist_ok=True, parents=True)

    if np.sum(w.annI[..., :3]) > 0:  # only save if something was masked
        plt.imsave(str(out_path), w.annI)
    else:
        # Annotation is empty, delete the corresponding file if it exists
        if os.path.exists(out_path):
            os.remove(out_path)

    progress.value += 1

    global curr_im
    global feats
    feats, curr_im = read_panel_image(site_zip_Path, progress.value, normalize=True)
    ann_path_curr = Path('figs') / 'annotations' / ('date_' + str(progress.value) + '_annotation.png')
    if os.path.exists(ann_path_curr):
        annotation = plt.imread(ann_path_curr)
    elif os.path.exists(out_path):
        annotation = plt.imread(out_path)
    else:
        annotation = None
        
    w.reset()
    w.set_image(curr_im, annotation, feats)

def previous(b):
    global w
    progress.value -= 1

    global curr_im
    global feats
    feats, curr_im = read_panel_image(site_zip_Path, progress.value, normalize=True)
    ann_path_curr = Path('figs') / 'annotations' / ('date_' + str(progress.value) + '_annotation.png')
    if os.path.exists(ann_path_curr):
        annotation = plt.imread(ann_path_curr)
    elif os.path.exists(out_path):
        annotation = plt.imread(out_path)
    else:
        annotation = None
    w.reset()
    w.set_image(curr_im, annotation, feats)

validate_button.on_click(validate)
previous_button.on_click(previous)

In [4]:
feats, curr_im = read_panel_image(site_zip_Path, progress.value, normalize=True)
s = ipyfan.FullSegmenter(iis_model, curr_im, curr_ref=None, feats=feats, layout_size=500)
w = s.w  # this is how we access internal variables
widgets.VBox([s.get_widget(), widgets.HBox([validate_button, previous_button, progress])])

/home/franchesoni/mine/creations/phd/projects/current/jupyter_segmentation_dev/env_ipyfan/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "uint8". A coerced copy has been created.
  warnings.warn(


        [0, 0, 0, 0],
        [0, 0, 0, 0],…